In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from modelArh import CarGameAgentDoubleMaybe,CombinedCarGameAgentMaybe  # Your custom model
from balance_it import load_and_balance_for_double,load_and_balance_for_double_maybe,load_only
from GATrainer import get_last_gen
from Functions import load_all_levels
from GATrainer import env_fn
import numpy as np
import time
import Functions
import pygame


/home/pfe-admin/komporess/Jedinica - Copy/clean/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:


# === Parameters ===
recordings_folder = "/home/pfe-admin/komporess/Jedinica - Copy/clean/clean-codes/recordings_replay"
batch_size = 64
epochs = 100
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gen = 0

ray_number = 7
parametri = 6
stanja = 4
n_inputs = parametri + stanja + 2 * ray_number

In [3]:


# # === Load and unpack data ===
# gas_brake, left_right = load_and_balance_for_double(recordings_folder)

# gb_states = [s for s, _ in gas_brake]
# gb_actions = [a for _, a in gas_brake]

# lr_states = [s for s, _ in left_right]
# lr_actions = [a for _, a in left_right]

# print(f"Loaded {len(gb_states)} gas/brake samples after balancing.")
# print(f"Loaded {len(lr_states)} left/right samples after balancing.")


# === Load and unpack data ===
import random

gas_brake, left_right = load_only(recordings_folder)

gb_states = [s for s, _ in gas_brake]
lr_states = [s for s, _ in left_right]

gb_actions = []
lr_actions = []

# Counters for balancing
gb_counts = [0, 0, 0]  # Gas, Brake, Neither
lr_counts = [0, 0, 0]  # Left, Right, Neither

# Process Gas/Brake actions
for _, action in gas_brake:
    if action[0]:
        gb_actions.append(0)
        gb_counts[0] += 1
    elif action[1]:
        gb_actions.append(1)
        gb_counts[1] += 1
    else:
        gb_actions.append(2)
        gb_counts[2] += 1

# Process Left/Right actions
for _, action in left_right:
    if action[0]:
        lr_actions.append(0)
        lr_counts[0] += 1
    elif action[1]:
        lr_actions.append(1)
        lr_counts[1] += 1
    else:
        lr_actions.append(2)
        lr_counts[2] += 1

print(f"Initial GB Counts: {gb_counts}")
print(f"Initial LR Counts: {lr_counts}")
# print(gb_actions[:10])
# --- Boost "Neither" Class ---

# def boost_class(states, actions, target_class, desired_total):
#     combined = list(zip(states, actions))
#     current_count = actions.count(target_class)

#     while current_count < desired_total:
#         for s, a in combined:
#             if a == target_class:
#                 states.append(s)
#                 actions.append(a)
#                 current_count += 1
#                 if current_count >= desired_total:
#                     break

# # Desired balancing: max(all classes) count for each class
# max_gb = max(gb_counts)
# max_lr = max(lr_counts)




# for i in range(3):
#     boost_class(lr_states, lr_actions, i, max_lr)
    

# Optional: shuffle after boosting
# gb_states = [s for s, _ in gas_brake]
# gb_actions = [a for _, a in gas_brake]




if lr_counts[0] > lr_counts[1]:
    diff = lr_counts[0] - lr_counts[1]
    minority_samples = [sample for sample in left_right if sample[1][1]]
    for _ in range(diff):
        left_right.append(random.choice(minority_samples))





combined_gb = list(zip(gb_states, gb_actions))
random.shuffle(combined_gb)
gb_states, gb_actions = zip(*combined_gb)


combined_lr = list(zip(lr_states, lr_actions))
random.shuffle(combined_lr)
lr_states, lr_actions = zip(*combined_lr)

# print(f"After Balancing - GB: {gb_actions.count(0)} Gas, {gb_actions.count(1)} Brake, {gb_actions.count(2)} Neither")
# print(f"After Balancing - LR: {lr_actions.count(0)} Left, {lr_actions.count(1)} Right, {lr_actions.count(2)} Neither")


print(f"Loaded {len(gb_states)} gas/brake samples after balancing.")
print(f"Loaded {len(lr_states)} left/right samples after balancing.")


bg_number=[0,0,0]

for action in gb_actions:
    bg_number[action]+=1

print(bg_number)

gb_actions=list(gb_actions)
gb_states=list(gb_states)

# print(gb_actions)


lr_number=[0,0,0]   

for action in lr_actions:
    lr_number[action]+=1

print(lr_number)

lr_actions=list(lr_actions)
lr_states=list(lr_states)





Initial GB Counts: [20935, 3550, 438]
Initial LR Counts: [3717, 4280, 16926]
Loaded 24923 gas/brake samples after balancing.
Loaded 24923 left/right samples after balancing.
[20935, 3550, 438]
[3717, 4280, 16926]


In [4]:
from ClassesML2 import vis_single_state_cv2
z=random.randint(0,100)
# for i in range(z+i,z+i+100):
#     print(gb_states[i],end="\n")

b=gb_states[z]
# inters=b[-14:]
# for i in range(len(inters)):
#     inters[i]=inters[i]=(2*inters[i]-1)*2000
print(gb_states[z])
print(len(gb_states[z]))
# print(inters)
# import cv2
# vis_single_state_cv2(7,b)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[0.05       0.2        0.2        0.25       0.04166667 0.15
 0.67655568 0.03220935 0.41262163 0.42395213 0.53894548 0.51210721
 0.51584611 0.57633524 0.56098793 0.56024966 0.5437374  0.51652407
 0.57587445 0.56048652 0.55895885 0.54337741 0.54353731 0.54164125
 0.05       0.2        0.2        0.25       0.04166667 0.15
 0.55533373 0.00307125 0.40954653 0.40718972 0.52376361 0.51266529
 0.5105625  0.58038178 0.56822466 0.55833547 0.5495043  0.51075978
 0.58128137 0.569212   0.56449336 0.55022274 0.54497611 0.54431499
 0.         1.         1.         0.         0.05       0.2
 0.2        0.25       0.04166667 0.15       0.58010067 0.00645782
 0.41239942 0.41011695 0.52594258 0.51285719 0.51039848 0.58024683
 0.56744263 0.55931559 0.54884753 0.51054551 0.58066485 0.56821505
 0.56344744 0.54940825 0.54472288 0.54406548 0.         1.
 0.         0.         0.05       0.2        0.2        0.25
 0.04166667 0.15       0.58010067 0.00645782 0.41525231 0.41304417
 0.52543264 0.51260446 0.510

In [5]:
# cv2.destroyAllWindows()